In [1]:
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect

import numpy as np
from numpy.random import default_rng

import thztools as thz

In [2]:
n = 256  # Number of samples
m = 50   # Number of traces
ts = 0.05  # Sampling time [ps]
t0 = 2.5  # Peak pulse time [ps]
fs = 1 / ts  # Sampling frequency [THz]

sigma_alpha = 1e-4  # Additive noise amplitude [signal units]
sigma_beta = 1e-2  # Multiplicative noise amplitude [dimensionless]
sigma_tau = 1e-3  # Time base noise amplitude [ps]
sigma_parms = np.array([sigma_alpha, sigma_beta, sigma_tau])

seed = 0  # RNG seed
rng = default_rng(seed)

In [3]:
mu, t = thz.thzgen(n, ts, t0)
sigma = thz.noiseamp(sigma_parms, mu, ts)

# Use broadcasting to generate multiple simulated noisy measurements of y
x = (mu + sigma * rng.standard_normal((m, n)))

In [4]:
# tdnoisefit expects a column-oriented data array
a0 = np.ones(m)
eta0 = np.zeros(m)
result = thz.tdnoisefit(x.T, a0=a0, eta0=eta0, ts=ts,
                        fix_a=True, fix_eta=True,
                        ignore_a=False, ignore_eta=False)

In [5]:
result[0]['var']

array([9.77331918e-09, 9.56912446e-05, 9.66479394e-07])

In [6]:
v0 = sigma_parms ** 2

In [7]:
v0

array([1.e-08, 1.e-04, 1.e-06])